# CulturalAssistant Bot

Этот блокнот содержит в себе все исходные файлы проекта и предназначен для запуска в Google Colab. Такое действие позволяет пользоваться GPU сервиса, необходимыми для запуска основной модели. Данный ноутбук предназначен для тестирования.

Здесь предполагается, что бот ещё не содержит базу данных с пользователями. Кроме того, некоторые файлы (индекс FAISS) находятся на Google Drive, а токен для telegram - в секретах Google Colab.

## Libraries

In [1]:
!pip install langchain
!pip install -qU langchain-community faiss-cpu
!pip install -qU langchain-huggingface

!pip install python-telegram-bot --upgrade
!pip install nest_asyncio

!pip install -U bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 38.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 33.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 46.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 26.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 40.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/

In [2]:
import torch
import sqlite3
from transformers import AutoTokenizer, AutoModelForCausalLM, GenerationConfig
from langchain.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from typing import List, Dict
from collections import OrderedDict

## Config

In [3]:
main_prompt = """
Твоя роль - это эксперт-консультант по подбору товаров для хобби.

Твоя задача - найти идеальные товары на основе истории диалога или уточнить предпочтения.

Входные данные:

История диалога: История взаимодействий.
Список товаров: Название, описание, ссылка.
Количество сообщений от меня: Число моих сообщений в истории диалога.

Выходные данные:
Если (Количество сообщений от меня < 3): Сгенерировать один релевантный и наводящий вопрос для уточнения хобби и предпочтений. Вопрос должен быть конкретным и направленным на получение полезной информации для подбора. Вопрос должен быть построен так, чтобы он был максимально полезен для определения стиля и предпочтений собеседника (например, для украшения подарка это может быть вопрос о том, в каком стиле должен быть украшен подарок, какие цвета предпочитает получатель и т.п.)

Иначе, если (Недостаточно информации для определения конкретных интересов ИЛИ Последнее сообщение от собеседника слишком общее и требует уточнения): Сгенерировать один релевантный и наводящий вопрос для уточнения хобби и предпочтений. Вопрос должен быть конкретным и направленным на получение полезной информации для подбора. Вопрос должен быть построен так, чтобы он был максимально полезен для определения стиля и предпочтений собеседника (например, для украшения подарка это может быть вопрос о том, в каком стиле должен быть украшен подарок, какие цвета предпочитает получатель и т.п.)

Иначе, если (Есть подходящие товары в списке):

Если несколько товаров, то выбрать 2 наиболее релевантных, используя конкретные названия, ссылки и описания:

Я думаю, вам могут подойти эти товары:
[Название товара 1]: [Описание товара 1] ([Ссылка на товар 1])
[Название товара 2]: [Описание товара 2] ([Ссылка на товар 2])
...
Если один товар, используя конкретные названия, ссылки и описания:

Мне кажется, вам отлично подойдет [Название товара]: [Описание товара] ([Ссылка на товар])

Если (Количество сообщений от меня > 5):

Выбрать наиболее подходящий товар в списке, используя конкретные названия, ссылки и описания:

Мне кажется, вам отлично подойдет [Название товара]: [Описание товара] ([Ссылка на товар])

Правила:

Действовать строго по алгоритму: Сначала проверить количество сообщений и достаточность информации. Затем искать подходящие товары.
Избегать лишних фраз и объяснений.
Быть конкретным: Использовать информацию из списка товаров и истории диалога.
Сохранять вежливый и полезный тон.
Быть максимально кратким.
Избегать слова ‘пользователь’.
При выводе товаров указывать конкретные названия, описания и ссылки.
При выводе товаров обязательно писать ссылку на этот товар.
При генерации вопроса учитывать, что последнее сообщение собеседника могло быть очень общим и требовать уточнения.
Важно: Сначала проверять количество сообщений. Если их меньше 3, независимо от содержания последнего сообщения, генерировать только вопросы.
Не генерировать ответ больше, чем на 100 слов.
Не задавай больше, чем 2 вопроса за раз.
Если количество сообщений от меня больше, чем 5, то независимо от содержания диалога выбирать товар из списка.
Выводи товары только из входного списка.
"""

In [4]:
config = {
    "chat_db_path": "users_messages.db",
    "faiss_db_path": "faiss_index",
    "start_message": "Привет, я ИИ ассистент, помогающий в выборе предметов для хобби. Напиши, какая область тебя интересует, и я подберу что-нибудь специально для тебя. \n\nЧтобы очистить контекст и начать выбор нового товара, пиши команду /clear_context.",
    "system_prompt": main_prompt
}

## Data Sources

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
def load_faiss_index(embeddings, path="faiss_index"):
    db = FAISS.load_local(
        path, embeddings, allow_dangerous_deserialization=True
    )
    return db


def load_user_db(path='users_messages.db'):
    connection = sqlite3.connect(path)
    cursor = connection.cursor()

    cursor.execute('''
    CREATE TABLE IF NOT EXISTS UsersMessages (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        user_id INTEGER NOT NULL,
        user_message TEXT NOT NULL,
        assistant_message TEXT NOT NULL,
        rag_addition TEXT NOT NULL,
        cleared INTEGER NOT NULL
    )
    ''')

    connection.commit()
    return connection

In [8]:
embeddings_for_faiss = HuggingFaceEmbeddings(model_name="BAAI/bge-m3", model_kwargs={'device': "cpu"})
user_messages_db = load_user_db(config["chat_db_path"])
faiss_db = load_faiss_index(embeddings_for_faiss, "drive/MyDrive/faiss_index")
# faiss_db = load_faiss_index(embeddings_for_faiss, config["faiss_db_path"])

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/15.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/444 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

## Models

In [9]:
class ModelWrapperBase:
    """ Base wrapper class that implements interface for models' responses """

    def __init__(self, system_prompt: str):
        """
        Initialization of parameters, tokenizers, embeddings and a sytem prompt

        Parameters
        ----------
        system_prompt : str
            System prompt for all answer generations.

        """
        pass

    def get_response(self, dialog: List[Dict], rag_information: str) -> str:
        """
        Model response generation conditioned by dialog history.

        Parameters
        ----------
        dialog : [{"role": ["user" | "assistant"], "content": str}, ...] list of dicts
            Representation of a dialog between model and user.
        rag_information: str
            RAG information for the last query of a dialog compressed in string

        Returns
        -------
        response : str
            Model response.

        """
        pass

In [10]:
class SaigaWrapper(ModelWrapperBase):
    """ Wrapper class for IlyaGusev/saiga_yandexgpt_8b """

    def __init__(self, system_prompt: str):
        super().__init__(system_prompt)

        self.system_prompt = system_prompt
        self.MODEL_NAME = "IlyaGusev/saiga_yandexgpt_8b"

        self.model = AutoModelForCausalLM.from_pretrained(
            self.MODEL_NAME,
            load_in_8bit=True,
            torch_dtype=torch.bfloat16,
            device_map="auto"
        )
        self.model.eval()

        self.tokenizer = AutoTokenizer.from_pretrained(self.MODEL_NAME)
        self.generation_config = GenerationConfig.from_pretrained(self.MODEL_NAME)

    def get_response(self, dialog: List[Dict], rag_information: str) -> str:

        messages = dialog.copy()

        print()
        print(rag_information)
        print()

        messages.append({"role": "system", "content": rag_information + "\n" + self.system_prompt})

        prompt = self.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

        data = self.tokenizer(prompt, return_tensors="pt", add_special_tokens=False)
        data = {k: v.to(self.model.device) for k, v in data.items()}
        data.pop("token_type_ids", None)
        output_ids = self.model.generate(**data, generation_config=self.generation_config, max_new_tokens=400)[0]
        output_ids = output_ids[len(data["input_ids"][0]):]
        output = self.tokenizer.decode(output_ids, skip_special_tokens=True).strip()

        torch.cuda.empty_cache()

        return output

In [11]:
model = SaigaWrapper(config["system_prompt"])

config.json:   0%|          | 0.00/747 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

...

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/276 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/190k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/18.0M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/210 [00:00<?, ?B/s]

## Bot Controller

In [12]:
# for async in Colab Notebook
import nest_asyncio
nest_asyncio.apply()

In [13]:
class BotController:

    def __init__(self, model, users_data, rag_data) -> None:
        self.model = model
        self.users_data = users_data
        self.rag_data = rag_data

    def load_dialog_and_rag_history(self, user_id: str):
        messages = self.users_data.cursor().execute(
            'SELECT user_message, assistant_message, rag_addition FROM UsersMessages WHERE (user_id = ?) AND (cleared = 0) ORDER BY id',
            (user_id,)
        ).fetchall()

        dialog_from_messages = []
        rag_items = []
        for (user_mes, assist_mes, rag_addition) in messages:
            dialog_from_messages.append({"role": "user", "content": str(user_mes)})
            dialog_from_messages.append({"role": "assistant", "content": str(assist_mes)})
            rag_items.extend(rag_addition.split('\n'))

        rag_items = list(OrderedDict.fromkeys(rag_items))

        return dialog_from_messages, rag_items

    def generate_answer(self, user_id: str, user_text: str) -> str:
        dialog_from_messages_history, rag_items_history = self.load_dialog_and_rag_history(user_id)

        dialog_from_messages_history.append({"role": "user", "content": str(user_text)})

        rag_query = "\n".join(
            [message["content"] for message in filter(lambda x: x["role"] == "user", dialog_from_messages_history)]
        )
        rag_items_history.extend(
            [
                f"Название товара: {str(doc.metadata['name'])}. Ссылка на товар: {str(doc.metadata['url'])}" for doc in
                self.rag_data.similarity_search(rag_query, k=5)
            ]
        )
        rag_items_history = rag_items_history[-10:]
        rag_info = "Выбранные товары:\n" + "\n".join(rag_items_history) + f"\n Количество сообщений от тебя: {(len(dialog_from_messages_history) - 1) // 2} \n"

        answer = self.model.get_response(
            dialog_from_messages_history,
            rag_info
        )

        self.create_note_in_source(user_id, user_text, answer, rag_items_history)
        return answer

    def create_note_in_source(self, user_id, user_text, answer, rag_items_history) -> None:
        self.users_data.cursor().execute(
            'INSERT INTO UsersMessages (user_id, user_message, assistant_message, rag_addition, cleared) VALUES (?, ?, ?, ?, ?)',
            (user_id, user_text, answer, "\n".join(rag_items_history), 0)
        )
        self.users_data.commit()

    def clear_context(self, user_id: str) -> None:
        self.users_data.cursor().execute(
            'UPDATE UsersMessages SET cleared = ? WHERE user_id = ?',
            (1, str(user_id))
        )
        self.users_data.commit()

## Telegram connection

In [14]:
from telegram import Update
from telegram.ext import ApplicationBuilder, CommandHandler, MessageHandler, filters, ContextTypes
import os

from google.colab import userdata
TOKEN = userdata.get('TOKEN')

if not TOKEN:
    raise ValueError("TOKEN missing")

bot_controller = BotController(model, user_messages_db, faiss_db)


async def start(update: Update, context: ContextTypes.DEFAULT_TYPE) -> None:
    """ Start-message reply """
    await update.message.reply_text(config["start_message"])


async def bot_response(update: Update, context: ContextTypes.DEFAULT_TYPE) -> None:
    """ User Message Processing  """
    user_id = str(update.message.from_user.id)
    text = str(update.message.text)

    if text:
        answer = bot_controller.generate_answer(user_id, text)
        await update.message.reply_text(answer)


async def clear_context(update: Update, context: ContextTypes.DEFAULT_TYPE) -> None:
    """ Clear assistant context """
    bot_controller.clear_context(str(update.message.from_user.id))
    await update.message.reply_text("Контекст обновлён")


def main() -> None:
    application = ApplicationBuilder().token(TOKEN).build()

    application.add_handler(CommandHandler("start", start))
    application.add_handler(CommandHandler("clear_context", clear_context))
    application.add_handler(MessageHandler(filters.TEXT & ~filters.COMMAND, bot_response))

    application.run_polling()


if __name__ == '__main__':
    main()

RuntimeError: Cannot close a running event loop